In [1]:
%%capture
!pip install lancedb embed-anything-gpu datasets huggingface_hub smolagents

In [5]:
import os
import requests
from urllib.parse import urlparse

urls = [
    "https://content.dgft.gov.in/Website/CIEP.pdf",
    "https://content.dgft.gov.in/Website/GAE.pdf",
    "https://content.dgft.gov.in/Website/HTE.pdf",
]

med_url = ["https://www.biorxiv.org/content/10.1101/2025.01.23.634433v1.full.pdf"]

def download_files(urls, folder_name="downloaded_docs"):
    """Downloads files from a list of URLs to a specified folder."""

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for url in urls:
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # Raise an exception for bad status codes

            parsed_url = urlparse(url)
            filename = os.path.basename(parsed_url.path)
            filepath = os.path.join(folder_name, filename)

            with open(filepath, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

            print(f"Downloaded {filename} to {folder_name}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading {url}: {e}")


def med_files(urls, folder_name="medical_docs"):
    """Downloads files from a list of URLs to a specified folder."""

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for url in med_url:
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # Raise an exception for bad status codes

            parsed_url = urlparse(url)
            filename = os.path.basename(parsed_url.path)
            filepath = os.path.join(folder_name, filename)

            with open(filepath, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

            print(f"Downloaded {filename} to {folder_name}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading {url}: {e}")

download_files(urls)
med_files(med_url)

Downloaded CIEP.pdf to downloaded_docs
Downloaded GAE.pdf to downloaded_docs
Downloaded HTE.pdf to downloaded_docs
Downloaded 2025.01.23.634433v1.full.pdf to medical_docs


In [16]:
import datasets
from smolagents import Tool

import lancedb
import embed_anything
from embed_anything import EmbeddingModel, WhichModel, ONNXModel
from uuid import uuid4


class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve policies about india that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, directory, **kwargs):
        super().__init__(**kwargs)
        self.model = EmbeddingModel.from_pretrained_onnx(WhichModel.Bert, ONNXModel.AllMiniLML6V2Q)
        self.connection = lancedb.connect("tmp/general")
        if "docs" in self.connection.table_names():
            self.table = self.connection.open_table("docs")
        else:
            self.embeddings = embed_anything.embed_directory(directory, embedder = self.model)
            docs = []
            for e in self.embeddings:
                docs.append({
                    "vector": e.embedding,
                    "text": e.text,
                    "id": str(uuid4())
                })
            self.table = self.connection.create_table("docs", docs)

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        query_vec = embed_anything.embed_query([query], embedder = self.model)[0].embedding
        docs = self.table.search(query_vec).limit(5).to_pandas()["text"]
        return "\nRetrieved documents:\n" + "".join(
            [f"\n\n===== Document {str(i)} =====\n" + doc for i, doc in enumerate(docs)]
        )

class MedicalRetrieverTool(Tool):
    name = "medical_retriever"
    description = "Uses semantic search to retrieve medicine related documennts most relevant to answer your query. Use this when the query is rlated to medicine or medical science or physiotherapy or psychology. "
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, directory, **kwargs):
        super().__init__(**kwargs)
        self.model =EmbeddingModel.from_pretrained_hf(WhichModel.Bert, model_id='NeuML/pubmedbert-base-embeddings')
        self.connection = lancedb.connect("tmp/medical")
        if "docs" in self.connection.table_names():
            self.table = self.connection.open_table("docs")
        else:
            self.embeddings = embed_anything.embed_directory(directory, embedder = self.model)
            docs = []
            for e in self.embeddings:
                docs.append({
                    "vector": e.embedding,
                    "text": e.text,
                    "id": str(uuid4())
                })
            self.table = self.connection.create_table("docs", docs)

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        query_vec = embed_anything.embed_query([query], embedder = self.model)[0].embedding
        docs = self.table.search(query_vec).limit(5).to_pandas()["text"]
        return "\nRetrieved documents:\n" + "".join(
            [f"\n\n===== Document {str(i)} =====\n" + doc for i, doc in enumerate(docs)]
        )

In [18]:
import datasets
from smolagents import Tool

import lancedb
import embed_anything
from embed_anything import EmbeddingModel, WhichModel, ONNXModel
from uuid import uuid4

# class RetrieverTool(Tool):
#     name = "retriever"
#     description = "Uses semantic search to retrieve policies about india that could be most relevant to answer your query."
#     inputs = {
#         "query": {
#             "type": "string",
#             "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
#         }
#     }
#     output_type = "string"

#     def __init__(self, directory, **kwargs):
#         super().__init__(**kwargs)
#         self.model = EmbeddingModel.from_pretrained_onnx(WhichModel.Bert, ONNXModel.AllMiniLML6V2Q)
#         self.connection = lancedb.connect("tmp/lancedb")
#         if "docs" in self.connection.table_names():
#             self.table = self.connection.open_table("docs")
#         else:
#             self.embeddings = embed_anything.embed_directory(directory, embedder = self.model)
#             docs = []
#             for e in self.embeddings:
#                 docs.append({
#                     "vector": e.embedding,
#                     "text": e.text,
#                     "id": str(uuid4())
#                 })
#             self.table = self.connection.create_table("docs", docs)

#     def forward(self, query: str) -> str:
#         assert isinstance(query, str), "Your search query must be a string"

#         query_vec = embed_anything.embed_query([query], embedder = self.model)[0].embedding
#         docs = self.table.search(query_vec).limit(5).to_pandas()["text"]
#         return "\nRetrieved documents:\n" + "".join(
#             [f"\n\n===== Document {str(i)} =====\n" + doc for i, doc in enumerate(docs)]
#         )


from smolagents import CodeAgent, OpenAIServerModel, TransformersModel, DuckDuckGoSearchTool
from google.colab import userdata
api_key = os.environ.get('OPENAI_API_KEY') # Get the key from environment variables
retriever_tool = RetrieverTool("downloaded_docs")
medical_tool = MedicalRetrieverTool("medical_docs")
# medical_tool = RetrieverTool("downloaded_docs")
agent = CodeAgent(
    tools=[retriever_tool, medical_tool],
    model=OpenAIServerModel(model_id = "gpt-4o-mini", api_base = "https://api.openai.com/v1/", api_key = api_key),
    verbosity_level=2,
)

agent_output = agent.run("What are the different policies for indian manufacturing and what are the medical risks of radiotherapy?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the different policies for indian manufacturing and what are the medical risks of radiotherapy?        │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will first gather information regarding the different policies for Indian manufacturing by using the    
`retriever` tool. After that, I will address the medical risks associated with radiotherapy using the              
`medical_retriever` tool.                                                                                          
                                                                                                                   
Let's begin by retrieving the policies for Indian manufacturing.                                                   
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
manufacturing_policies = retriever(query="Indian manufacturing policies")                                          
print(manufacturing_policies)                                                                                      
```                                                                                                                

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  manufacturing_policies = retriever(query="Indian manufacturing policies")                                        
  print(manufacturing_policies)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
62 | P a g e       The  National  Manufacturing  Policy  of  the  Government  of  India  aims  to  increase  the 
share of manufacturing in GDP to 25 percent by 2025.      The “Make in India” policy of the Government of India is
expected to increase the  demand  and  consumption  for  machinery  and  equipment  by  the  local  manufacturing 
industry.      To be implemented from 2022, the Production-linked incentive (PLI) scheme was  launched  by  the  
Government  of  India  to  develop  13  sectors  on  par  with  global  manufacturing standards.      India is 
gradually progressing on the road to Industry 4.0 through the  Government of India’s initiatives.    Government of 
India has undertaken several schemes/initiatives to promote  manufacturing sector.    Schemes undertaken by DPIIT 
to promote manufacturing sector:    Government  of  India  has  undertaken  various  steps  to  promote  
manufacturing  sector  and  to  boost  domestic  and  foreign  investments  in  India.

===== Document 1 =====
The  EOUs  function  under  the  administrative  control  of  the  concerned  Development  Commissioner  of  
Special  Economic  Zone  i.e.,  under  the  Department of Commerce, Government of India.  The  EOUs  are  governed 
by  the  provisions  of  Chapter  6  of  the  Foreign  Trade  Policy  (FTP) and its procedures, as contained in the
Handbook of Procedures (HBH).  MANUFACTURING / PRODUCING EXPORTS  Introduction:      The  Indian  manufacturing  
industry  generated  16-17  percent  of  India’s  GDP  pre-pandemic and is projected to be  one of the fastest 
growing sectors.  The machine  tool industry was literally the nuts and bolts of the manufacturing industry in 
India.  Today, technology has stimulated innovation with digital transformation a key aspect  in gaining an edge in
this highly competitive market.      Both  the  Government  of  India  and  Indian  companies  recognizes  the  
need  to  move forward to the next and latest phase of industry.

===== Document 2 =====
National Logistics Policy:     National  Logistics  Policy  (NLP)  was  launched  on  17th  September  2022,  that 
aims to lower the cost of logistics and lead it to par with other developed countries.  It  is  a  comprehensive  
effort  to  address  cost  inefficiency  by  laying  down  an  overarching interdisciplinary, cross-sectoral, and 
multi-jurisdictional framework for    64 | P a g e     developing  entire  logistics  ecosystem.  This  would  
boost  economic  growth,  provide  employment  opportunities,  and  make  Indian  products  more  competitive  in  
the  global market.  Production Linked Incentive scheme:     Keeping  in  view  India’s  vision  of  becoming  
‘Atmanirbhar’,  Production  Linked  Incentive (PLI) Schemes for 14 key sectors have been announced with an outlay 
of  Rs.  1.97  lakh  crore  to  enhance  India’s  manufacturing  capabilities  and  Exports.

===== Document 3 =====
Further, exemption from the additional duties of customs, if any, under section  3(1), section 3(3) and section 
3(5) of the Customs Tariff Act, 1975 and exemption  from central excise duty will be available for goods specified 
under the Fourth  Schedule to the Central Excise Act.                         GST & Exports  Page 12       DEEMED 
EXPORTS    As  per  the  Foreign  Trade  Policy,  Deemed  Exports  refer  to  those  transactions  in  which  goods
supplied  do  not  leave  the  country,  and  payment  is  realized  either  in  Indian currency or free foreign 
exchange. The precise categories of deemed exports  are defined in Para 7.02 of the FTP.    “Deemed Exports” refers
to supplies of goods manufactured in India (and not  services)  which  are  notified  as  deemed  exports  under  
Section  147  of  the  CGST/SGST  Act,  2017.  The  supplies  do  not  leave  India.

===== Document 4 =====
  To  boost  merchanting  activities  from  India  –  where  goods  do  not  touch  I

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I have retrieved various documents concerning the medical risks associated with radiotherapy. Key findings
from the documents include:                                                                                        
                                                                                                                   
1. **Radiotherapy Treatment**: It's a common cancer treatment that can lead to increased tumor resistance due to   
the emergence of cancer stem cells (CSCs).                                                                         
2. **DNA Damage**: Ionizing radiation kills cancer cells through direct DNA disruption or via reactive oxygen      
species causing indirect DNA damage.                                                                               
3. **Fractionated Radiation**: It is typically given in fractions to minimize harm to normal tissues, with various 
methods (conventional, hyperfractionation, hypofractionation) affecting the treatment outcomes.                    
4. **Radioresistance**: There are indications that cancer cells can adapt and develop resistance during treatment, 
potentially complicating effective management.                                                                     
                                                                                                                   
Now I have sufficient information on both the Indian manufacturing policies and the medical risks of radiotherapy. 
I will summarize these findings and provide the final answer.                                                      
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer({                                                                                                     
    "Indian Manufacturing Policies": [                                                                             
        "National Manufacturing Policy: Aims to increase the share of manufacturing in GDP to 25% by 2025.",       
        "'Make in India' Policy: Expected to boost machinery and equipment demand by local manufacturers.",        
        "Production-linked Incentive (PLI) Scheme: Launched to enhance manufacturing capabilities in 13 sectors.", 
        "National Logistics Policy: Aims to lower logistics costs and improve global competitiveness.",            
        "Deemed Exports: Refers to goods that do not leave India but still count as exports."                      
    ],                                                                                                             
    "Medical Risks of Radiotherapy": [                                                                             
        "Increases tumor resistance and proliferation rates due to cancer stem cells (CSCs).",                     
        "Ionizing radiation can cause direct DNA damage or indirect damage via reactive oxygen species.",          
        "Fractionation methods (such as conventional, hyperfractionation, and hypo

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Indian Manufacturing Policies": [                                                                           
          "National Manufacturing Policy: Aims to increase the share of manufacturing in GDP to 25% by 2025.",     
          "'Make in India' Policy: Expected to boost machinery and equipment demand by local manufacturers.",      
          "Production-linked Incentive (PLI) Scheme: Launched to enhance manufacturing capabilities in 13          
  sectors.",                                                                                                       
          "National Logistics Policy: Aims to lower logistics costs and improve global competitiveness.",          
          "Deemed Exports: Refers to goods that do not leave India but still count as exports."                    
      ],                                                                                                           
      "Medical Risks of Radiotherapy": [                                                                           
          "Increases tumor resistance and proliferation rates due to cancer stem cells (CSCs).",                   
          "Ionizing radiation can cause direct DNA damage or indirect damage via reactive oxygen species.",        
          "Fractionation methods (such as conventional, hyperfractionation, and hypofractionation) are used to     
  limit normal tissue damage.",                                                                                    
          "Cancer cells can develop radioresistance during treatment, complicating management."                    
      ]                                                                                                            
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Indian Manufacturing Policies': ['National Manufacturing Policy: Aims to increase the share 
of manufacturing in GDP to 25% by 2025.', "'Make in India' Policy: Expected to boost machinery and equipment demand
by local manufacturers.", 'Production-linked Incentive (PLI) Scheme: Launched to enhance manufacturing capabilities
in 13 sectors.', 'National Logistics Policy: Aims to lower logistics costs and improve global competitiveness.', 
'Deemed Exports: Refers to goods that do not leave India but still count as exports.'], 'Medical Risks of 
Radiotherapy': ['Increases tumor resistance and proliferation rates due to cancer stem cells (CSCs).', 'Ionizing 
radiation can cause direct DNA damage or indirect damage via reactive oxygen species.', 'Fractionation methods 
(such as conventional, hyperfractionation, and hypofractionation) are used to limit normal tissue damage.', 'Cancer
cells can develop radioresistance during treatment, complicating management.']}

[Step 2: Duration 9.57 seconds| Input tokens: 10,693 | Output tokens: 691]

Final output:
{'Indian Manufacturing Policies': ['National Manufacturing Policy: Aims to increase the share of manufacturing in GDP to 25% by 2025.', "'Make in India' Policy: Expected to boost machinery and equipment demand by local manufacturers.", 'Production-linked Incentive (PLI) Scheme: Launched to enhance manufacturing capabilities in 13 sectors.', 'National Logistics Policy: Aims to lower logistics costs and improve global competitiveness.', 'Deemed Exports: Refers to goods that do not leave India but still count as exports.'], 'Medical Risks of Radiotherapy': ['Increases tumor resistance and proliferation rates due to cancer stem cells (CSCs).', 'Ionizing radiation can cause direct DNA damage or indirect damage via reactive oxygen species.', 'Fractionation methods (such as conventional, hyperfractionation, and hypofractionation) are used to limit normal tissue damage.', 'Cancer cells can develop radioresistance during treatment, complicating management.']}
